In [ ]:
import sys 
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.rosetta.protocols.carbohydrates import *
from pyrosetta.rosetta.core.pose import *
from pyrosetta.rosetta.core.scoring import *
from pyrosetta.rosetta.protocols.minimization_packing import *
from pyrosetta.rosetta.core.pose.carbohydrates import glycosylate_pose, glycosylate_pose_by_file
from pyrosetta.rosetta.protocols.carbohydrates import SimpleGlycosylateMover
from pyrosetta.rosetta.protocols.rosetta_scripts import *

## Personal Imports
import basePeptidePrep
import addBaseSugarAndEnzyme

## Pyrosetta Initialization
pyrosetta.init('-include_sugars -write_pdb_link_records -ex1 -ex2 -alternate_3_letter_codes pdb_sugar -auto_detect_glycan_connections -min_bond_length 1.1 -max_bond_length 1.5 -ignore_zero_occupancy false -ignore_unrecognized_res false')

## Command Line Arguments 
'''
Command Line Arguments:
# For the base peptide 
- Sequence of the base peptide (2-3 AA long peptides are ideal) : base_seq
- Position of the sequence containing THR that will be glycosylated : base_pos
- the sugar that will be glycosylated on the base peptide : base_sugar

# input pdb files
- The enzyme with donor peptide 

- The constraint file

'''
base_seq = "STP"
base_position = 2
base_sugar = "core1"
input_enzyme_file = "/home/souvadra/myGitFolders/Glycosyltransferase/glycan_sampler_pipeline/output_3OTK-closed-S217C/3OTK-closed-monomer-alpha-GlcNAc-S217C_0005.pdb"
constraints_file = "constraints_file.cst"
#constraints_file = "trialConstraint.cst"
## Prepare the base peptide 
base_pose = basePeptidePrep.basePeptideBuild(base_seq, base_position, base_sugar)
print(base_pose)

## Addition of peptide and sugar motif 
enzyme_pose = pose_from_pdb(input_enzyme_file)

In [ ]:
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.rosetta.protocols.carbohydrates import *
from pyrosetta.rosetta.core.pose import *
from pyrosetta.rosetta.core.scoring import *
from pyrosetta.rosetta.protocols.minimization_packing import *
from pyrosetta.rosetta.core.pose.carbohydrates import glycosylate_pose, glycosylate_pose_by_file
from pyrosetta.rosetta.protocols.carbohydrates import SimpleGlycosylateMover
from pyrosetta.rosetta.protocols.rosetta_scripts import *

from pyrosetta.rosetta.core.scoring.constraints import *
from pyrosetta.rosetta.core.select.movemap import MoveMapFactory

def mergePoses(base_pose, enzyme_pose):
    """merge the base pose into enzyme pose without making any covalent bond"""
    anchor_point = int(enzyme_pose.total_residue()) - 2 # just to make sure that the UDP-sugar remains the last residues in the whole pose
    enzyme_pose.append_pose_by_jump(base_pose, anchor_point)

""" This program takes the base sugar pose and the pose of the enzyme with donor moiety
     and the constraints and make sure the they obey the constraint in subsequent relax
       procedure as a procedure to bypass manual overlaying using PyMOL. """
init("-constraints:cst_fa_file " + constraints_file)

sfxn = pyrosetta.create_score_function("ref2015_cart.wts") #get_fa_scorefxn()
initial_energy = sfxn(enzyme_pose)
# merging two poses to form one 
mergePoses(base_pose, enzyme_pose)

sfxn.set_weight(atom_pair_constraint, 1.0)
add_fa_constraints_from_cmdline(enzyme_pose, sfxn)
print(sfxn.show(enzyme_pose))

## Change the FoldTree to more Docking friendly option as suggested by Pooja 
first_residue_enzyme = 1
last_residue_enzyme = 369
anchor_residue_enzyme = 130
anchor_residue_substrate = 373
first_residue_substrate = 372
last_residue_substrate = 374
first_donor = 370
last_donor = 371

ft_docking = pyrosetta.FoldTree()
ft_docking.add_edge(anchor_residue_enzyme,1,-1)
ft_docking.add_edge(anchor_residue_enzyme,last_residue_enzyme,-1)

ft_docking.add_edge(anchor_residue_enzyme, anchor_residue_substrate, 1) 
ft_docking.add_edge(anchor_residue_enzyme, first_donor, 2)
            
ft_docking.add_edge(anchor_residue_substrate, first_residue_substrate, -1) # downstream base peptide 
ft_docking.add_edge(anchor_residue_substrate, last_residue_substrate, -1) # upstream base peptide 
ft_docking.add_edge(first_donor,last_donor,-1) # the donor UDP-GlcNAc backbone

chemical_edges = enzyme_pose.fold_tree().get_chemical_edges() # connections to the glycan residues from anchor out 
for chemical in range(1, len(chemical_edges) + 1):
    ft_docking.add_edge(chemical_edges[chemical].start(), chemical_edges[chemical].stop(), -2)

ft_docking.add_edge(375,376,-1) # Adding the covalent bond between the two sugars of core 1 
print(enzyme_pose.fold_tree())
enzyme_pose.fold_tree(ft_docking)
print(enzyme_pose.fold_tree())
    
    ## setting the movemap 
mm = pyrosetta.rosetta.core.kinematics.MoveMap()
mm.set_bb(False)
mm.set_chi(False)
mm.set_jump(True)

referencePose = pose_from_pdb("/home/souvadra/myGitFolders/Glycosyltransferase/Acceptor-Donor-Enzyme/GlcNAc-added-before-GalBGalNAc/3OTK-closed-monomer-alpha-GlcNAc_2GAM-GalBGalNAc.pdb")  
RMSD_list = []
score_list = []

In [ ]:
    for trial_number in range(0,20):
        ## Use Rigid Body Mover to bring the base peptide close to the enzyme
        curr_enzyme_pose = enzyme_pose.clone()
        rigid_body = pyrosetta.rosetta.protocols.rigid.RigidBodyPerturbNoCenterMover()
        rigid_body.add_jump(1)
        rigid_body.rot_magnitude(10)
        rigid_body.trans_magnitude(5.0)
        
        first_pose = curr_enzyme_pose.clone()
        num_iter = 0
        while (sfxn(curr_enzyme_pose) > 0 ):
            num_iter += 1
            rigid_body.apply(first_pose)
            if (sfxn(first_pose) < sfxn(curr_enzyme_pose)):
                curr_enzyme_pose = first_pose.clone()
                print(sfxn(curr_enzyme_pose))
            else:
                first_pose = curr_enzyme_pose.clone()
            if (num_iter > 500):
                break
            #print(sfxn(enzyme_pose))
        sfxn.show(curr_enzyme_pose)
        print(curr_enzyme_pose.fold_tree())

        rigid_body2 = pyrosetta.rosetta.protocols.rigid.RigidBodyPerturbNoCenterMover()
        rigid_body2.add_jump(1)
        rigid_body2.rot_magnitude(0)
        rigid_body2.trans_magnitude(0.1)
        
        rigid_body_rot = pyrosetta.rosetta.protocols.rigid.RigidBodyPerturbNoCenterMover()
        rigid_body_rot.add_jump(1)
        rigid_body_rot.rot_magnitude(1)
        rigid_body_rot.trans_magnitude(0)
        
        first_pose = curr_enzyme_pose.clone()
        num_iter = 0
        while (sfxn(curr_enzyme_pose) > (initial_energy) ):
            num_iter += 1
            rigid_body_rot.apply(first_pose)
            if (sfxn(first_pose) < sfxn(curr_enzyme_pose)):
                curr_enzyme_pose = first_pose.clone()
                print(sfxn(curr_enzyme_pose), "  rotation-iteration ")    
            rigid_body2.apply(first_pose)
            if (sfxn(first_pose) < sfxn(curr_enzyme_pose)):
                curr_enzyme_pose = first_pose.clone()
                print(sfxn(curr_enzyme_pose), "  translation-iteration ")
            else:
                first_pose = curr_enzyme_pose.clone()
            print(num_iter)
            if (num_iter > 500):
                break
            #print(sfxn(enzyme_pose))
        score_list.append(sfxn(curr_enzyme_pose))
        RMSD_list.append(CA_rmsd(curr_enzyme_pose,referencePose))
        dumping_name = "merging_result" + str(trial_number) + ".pdb"
        curr_enzyme_pose.dump_pdb(dumping_name)

In [ ]:
import matplotlib.pylab as plt
plt.scatter(RMSD_list,score_list)
plt.xlabel("RMSD-value")
plt.ylabel("REU score")

In [ ]:
score_list

In [ ]:
CA_rmsd(enzyme_pose,referencePose)

In [ ]:
initial_energy